In [1]:
import numpy as np
import matplotlib.pyplot as plt
from neuralop.models import FNO
import torch
import matplotlib.pyplot as plt
import sys
from neuralop.models import TFNO
from neuralop import Trainer
from neuralop.datasets import load_darcy_flow_small
from neuralop.utils import count_params
from neuralop import LpLoss, H1Loss

device = 'cpu'

In [2]:
import torch
from neuralop.utils import UnitGaussianNormalizer
from neuralop.datasets.tensor_dataset import TensorDataset
from neuralop.datasets.transforms import PositionalEmbedding


def load_dataset(train_path, test_path,
                batch_size, test_batch_sizes,
                grid_boundaries=[[0,1],[0,1]],
                positional_encoding=False,
                encode_input=False,
                encode_output=False,
                encoding='pixel-wise', 
                channel_dim=1):
    """Loads a small Darcy-Flow dataset
    
    Training contains 1000 samples in resolution 16x16. 
    Testing contains 100 samples at resolution 16x16 and
    50 samples at resolution 32x32.

    Parameters
    ----------
    n_train : int
    n_tests : int
    batch_size : int
    test_batch_sizes : int list
    test_resolutions : int list, default is [16, 32],
    grid_boundaries : int list, default is [[0,1],[0,1]],
    positional_encoding : bool, default is True
    encode_input : bool, default is False
    encode_output : bool, default is True
    encoding : 'channel-wise'
    channel_dim : int, default is 1
        where to put the channel dimension, defaults size is batch, channel, height, width

    Returns
    -------
    training_dataloader, testing_dataloaders

    training_dataloader : torch DataLoader
    testing_dataloaders : dict (key: DataLoader)
    """
    # for res in test_resolutions:
    #     if res not in [16, 32]:
    #         raise ValueError(f'Only 32 and 64 are supported for test resolution, but got {test_resolutions=}')
    # path = Path(__file__).resolve().parent.joinpath('data')
    return load_dataset_pt(train_path, test_path, 
                         batch_size=batch_size, test_batch_size=test_batch_sizes,
                         grid_boundaries=grid_boundaries,
                         positional_encoding=positional_encoding,
                         encode_input=encode_input,
                         encode_output=encode_output,
                         encoding=encoding,
                         channel_dim=channel_dim)

def load_dataset_pt(train_data_path, test_data_path,
                batch_size, test_batch_size,
                grid_boundaries=[[0,1],[0,1]],
                positional_encoding=False,
                encode_input=False,
                encode_output=True,
                encoding='channel-wise', 
                channel_dim=1):
    """Load the Navier-Stokes dataset
    """
    data = np.load(train_data_path)
    # n_train = data.shape[0]
    x_train = data[:, 0]
    y_train = data[:, 1]
    del data

    data = np.load(test_data_path)
    # n_test = data.shape[0]
    x_test = data[:, 0]
    y_test = data[:, 1]
    del data
    
    # if encode_input:
    #     if encoding == 'channel-wise':
    #         reduce_dims = list(range(x_train.ndim))
    #     elif encoding == 'pixel-wise':
    #         reduce_dims = [0]

    #     input_encoder = UnitGaussianNormalizer(x_train, reduce_dim=reduce_dims)
    #     x_train = input_encoder.encode(x_train)
    #     x_test = input_encoder.encode(x_test.contiguous())
    # else:
    #     input_encoder = None

    # if encode_output:
    #     if encoding == 'channel-wise':
    #         reduce_dims = list(range(y_train.ndim))
    #     elif encoding == 'pixel-wise':
    #         reduce_dims = [0]

    #     output_encoder = UnitGaussianNormalizer(y_train, reduce_dim=reduce_dims)
    #     y_train = output_encoder.encode(y_train)
    # else:
    #     output_encoder = None
    output_encoder = None

    train_db = TensorDataset(x_train, y_train, transform_x=PositionalEmbedding(grid_boundaries, 0) if positional_encoding else None)
    train_loader = torch.utils.data.DataLoader(train_db,
                                            batch_size=batch_size, shuffle=True,
                                            num_workers=0, pin_memory=True, persistent_workers=False)

    test_db = TensorDataset(x_test, y_test,transform_x=PositionalEmbedding(grid_boundaries, 0) if positional_encoding else None)
    test_loader = torch.utils.data.DataLoader(test_db,
                                              batch_size=test_batch_size, shuffle=False,
                                              num_workers=0, pin_memory=True, persistent_workers=False)
    test_loaders =  {'1': test_loader}
    # for ( n_test, test_batch_size) in zip(n_test, test_batch_sizes):
    #     print(f'Loading test db with {n_test} samples and batch-size={test_batch_size}')
    #     data = torch.load(Path(data_path).joinpath(f'darcy_test_{res}.pt').as_posix())
    #     x_test = data['x'][:n_test, :, :].unsqueeze(channel_dim).type(torch.float32).clone()
    #     y_test = data['y'][:n_test, :, :].unsqueeze(channel_dim).clone()
    #     del data 
    #     if input_encoder is not None:
    #         x_test = input_encoder.encode(x_test)

    #     test_db = TensorDataset(x_test, y_test, transform_x=PositionalEmbedding(grid_boundaries, 0) if positional_encoding else None)
    #     test_loader = torch.utils.data.DataLoader(test_db,
    #                                               batch_size=test_batch_size, shuffle=False,
    #                                               num_workers=0, pin_memory=True, persistent_workers=False)
    #     test_loaders[res] = test_loader

    return train_loader, test_loaders, output_encoder


In [ ]:
train_path = '../data/input_train.npy'
train_loader, test_loaders, output_encoder = load_dataset(,)

In [3]:
model = FNO(n_modes=(16, 16), hidden_channels=64,
                in_channels=3, out_channels=1)
model = model.to(device)

n_params = count_params(model)
print(f'\nOur model has {n_params} parameters.')
sys.stdout.flush()


Our model has 4228097 parameters.


In [5]:
optimizer = torch.optim.Adam(model.parameters(), 
                                lr=8e-3, 
                                weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)

l2loss = LpLoss(d=1, p=2)
h1loss = H1Loss(d=1)

train_loss = h1loss
eval_losses={'h1': h1loss, 'l2': l2loss}

print('\n### MODEL ###\n', model)
print('\n### OPTIMIZER ###\n', optimizer)
print('\n### SCHEDULER ###\n', scheduler)
print('\n### LOSSES ###')
print(f'\n * Train: {train_loss}')
print(f'\n * Test: {eval_losses}')
sys.stdout.flush()


### MODEL ###
 FNO(
  (fno_blocks): FNOBlocks(
    (convs): FactorizedSpectralConv(
      (weight): ModuleList(
        (0-7): 8 x ComplexDenseTensor(shape=torch.Size([64, 64, 8, 8]), rank=None)
      )
    )
    (fno_skips): ModuleList(
      (0-3): 4 x Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    )
  )
  (lifting): Lifting(
    (fc): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
  )
  (projection): Projection(
    (fc1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
    (fc2): Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1))
  )
)

### OPTIMIZER ###
 Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    initial_lr: 0.008
    lr: 0.008
    maximize: False
    weight_decay: 0.0001
)

### SCHEDULER ###

### LOSSES ###

 * Train: <neuralop.training.losses.H1Loss object at 0x000001D468F573A0>

 * Test: {'h1': <neuralop.training.losses.H1Loss obj

In [20]:


trainer = Trainer(model, n_epochs=20,
                  device=device,
                  mg_patching_levels=0,
                  wandb_log=False,
                  log_test_interval=3,
                  use_distributed=False,
                  verbose=True)

trainer.train(input_train, input_test,
              None,
              model, 
              optimizer,
              scheduler, 
              regularizer=False, 
              training_loss=train_loss,
              eval_losses=eval_losses)

Training on regular inputs (no multi-grid patching).


AttributeError: 'numpy.ndarray' object has no attribute 'dataset'